# Tutorial for Analyzing LMS Files

## Purpose
This notebook is designed to help analysts understand and access the files created by the Ed-Fi LMS Toolkit extractor utilities.

## Pandas

We'll be using [Pandas](https://pandas.pydata.org/), which is the industry standard tool for high performance data analysis in Python. If you're new to Python and Pandas and want to do some additional reading, then you might be interested in these two books by Jake VanderPlas, both available for free:

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/)
* [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)

## Getting Started

This notebook was developed with Python 3.8 and might not work with earlier versions. The source code directory for this notebook contains a `poetry.lock` and `pyproject.toml` file; these are used by [Poetry](https://python-poetry.org/) to manage resource dependencies. Ideally you will install Poetry and then run `poetry install` from a command prompt to load required resources. If you would like to run without using Poetry, then you can manually load dependencies using Pip:

```bash
# Optional step if NOT using poetry
pip install pandas
pip install ipykernel
pip install jupyter --user
```

It is recommended that these be installed in a virtual environment, which Poetry handles for you. Alternately, you can use equivalent commands in Anaconda.

This notebook can be run from within Visual Studio Code or it can run in a browser window by executing the following command:

```bash
# With Poetry
poetry run jupyter notebook

# Without Poetry
jupyter notebook
```

## Understanding the Filesystem

The LMS Extractors output a number of discrete files, corresponding to concepts in the Ed-Fi LMS Unified Data Model:

* Activities
* Assignments
* Attendance (_only with Schoology_)
* Grades
* Sections
* Section Associations (aka _enrollments_)
* Submissions
* Users

Each file contains all of the current data for the given model, so that you only need to read one file to get a complete snapshot for a single resource. But there is a catch to that: some of these concepts are section-specific, for example assignments. Rather than try to store all assignment data _for all sections_ in a single file, we create _one assignment file per section_. The same is true for activities, attendance, grades, and section associations. Furthermore, submissions are dependent on assignments, thus there is one submissions file per assignment. The file layout mirrors this heirarchy: there is a directory for each resource type, and dependent resources are nested under directories named for the given section or assignment.

This convention may seem a little strange for a human, but it is very easy to navigate for the computer. Each directory may have multiple files, one for each time you run the extractor, but you only need to load the most recent file to get the current snapshot. We make that easy by using the date and time as the file name. Thus after running once, you may end up with files like this, where 12345 is the source system identifier for a unique section and 67890 is the source system identifier for a unique assignment:

![Sample file layout](filesystem.svg)

Note: `base_directory` is whatever directory was specified in the configuration when running the extractor utility.

Presumably the extractor utility will be run on a periodic basis, for instance weekly or daily. In that case each directory will have multiple files. Since the filenames have the date and time embedded in them, sorting on the file names will make it easy to pick up only the most recent file, regardless of whether or not some other process has modified the file and thus altered the operating system date on the file.

## Helper Functions

### Accessing Files

Below you will find a set of functions to help you navigate this filesystem.

LANGUAGE USE NOTE: in these examples we use Python's optional [type hint system](https://docs.python.org/3.8/library/typing.html) to help us all recognize what data types are being passed into functions and returned by them.

In [6]:
import os

def _get_newest_file(directory: str) -> str:
    if not os.path.exists(directory):
        return ""

    files = [(f.path, f.name) for f in os.scandir(directory) if f.name.endswith(".csv")]
    files = sorted(files, key=lambda x: x[1], reverse=True)

    return files[0][0]

def _get_file_for_section(base_directory: str, section_id: int, file_type: str) -> str:
    return _get_newest_file(os.path.join(base_directory, f"section={section_id}", file_type))

def get_users_file(base_directory: str) -> str:
    return _get_newest_file(os.path.join(base_directory, "users"))

def get_sections_file(base_directory: str) -> str:
    return _get_newest_file(os.path.join(base_directory, "sections"))

def get_section_associations_file(base_directory: str, section_id: int) -> str:
    return _get_file_for_section(base_directory, section_id, "section-associations")

def get_activities_file(base_directory: str, section_id: int) -> str:
    return _get_file_for_section(base_directory, section_id, "activities")

def get_assignments_file(base_directory: str, section_id: int) -> str:
    return _get_file_for_section(base_directory, section_id, "assignments")

def get_grades_file(base_directory: str, section_id: int) -> str:
    return _get_file_for_section(base_directory, section_id, "grades")
    
def get_submissions_file(base_directory: str, section_id: int, assignment_id: int) -> str:
    path = os.path.join(base_directory, f"section={section_id}", f"assignment={assignment_id}", "submissions")
    return _get_newest_file(path)

def get_attendance_events_file(base_directory: str, section_id: int) -> str:
    return _get_file_for_section(base_directory, section_id, "attendance-events")

### Loading Files into DataFrames

Next, let's create a few functions that leverage the filesystem helpers to read files into Pandas DataFrames. Note that the functions for data nested under sections is using the output from `get_all_sections`, and thus _will only pull data for active sections_. Thus if assignments, activities, etc. for section 12345 were retrieved historically, but section 12345 is no longer active (not in the current sections file), then these functions will ignore the data for section 12345. You can of course adapt these functions for your own use to access historical data as needed.

| Function Name | Arguments | Output |
| ------------- | --------- | ------ |
| get_all_users | base_directory (string) | DataFrame with all user records from the most recent file |
| get_all_sections | base_directory (string) | DataFrame with all section records from the most recent file |
| get_section_associations | base_directory (string), section_id (int) | DataFrame with all section associations for the given section_id |
| get_all_section_associations | base_directory (string), sections (DataFrame) | DataFrame with all section associations from all sections |
| get_activities | base_directory (string), section_id (int) | DataFrame with all activities for the given section_id |
| get_all_activities | base_directory (string), sections (DataFrame) | DataFrame with all activities from all sections |
| get_assignments | base_directory (string), section_id (int) | DataFrame with all assignments for the given section_id |
| get_all_assignments | base_directory (string), sections (DataFrame) | DataFrame with all assignments from all sections |
| get_attendance_events | base_directory (string), section_id (int) | DataFrame with all attendance events for the given section_id |
| get_all_attendance_events | base_directory (string), sections (DataFrame) | DataFrame with all attendance events from all sections |
| get_grades | base_directory (string), section_id (int) | DataFrame with all grades for the given section_id |
| get_all_grades | base_directory (string), sections (DataFrame) | DataFrame with all grades from all sections |
| get_submissions | base_directory (string), section_id (int), assignment_id(int) | DataFrame with all submissions for the given assignment |
| get_all_submissions | base_directory (string), assignments (DataFrame) | DataFrame with all submissions from all sections |

In [7]:
from typing import Callable
import pandas as pd  # Aliasing as `pd` is a common practice in the industry

def _read_csv(file: str) -> pd.DataFrame:
    if file:
        return pd.read_csv(file, engine="c", parse_dates=True, infer_datetime_format=True)

    return pd.DataFrame()

def get_all_users(base_directory: str) -> pd.DataFrame:
    file = get_users_file(base_directory)

    return _read_csv(file)
    
def get_all_sections(base_directory: str) -> pd.DataFrame:
    file = get_sections_file(base_directory)

    return _read_csv(file)

def get_section_associations(base_directory: str, section_id: int) -> pd.DataFrame:
    file = get_section_associations_file(base_directory, section_id)

    return _read_csv(file)

def _get_data_for_section(base_directory: str, sections: pd.DataFrame, callback: Callable) -> pd.DataFrame:
    df = pd.DataFrame()
    for _, section_id in sections[["SourceSystemIdentifier"]].itertuples():
        df = df.append(callback(base_directory, section_id))

    return df

def get_all_section_associations(base_directory: str, sections: pd.DataFrame) -> pd.DataFrame:    
    return _get_data_for_section(base_directory, sections, get_section_associations)

def get_activities(base_directory: str, section_id: int) -> pd.DataFrame:
    file = get_activities_file(base_directory, section_id)

    return _read_csv(file)

def get_all_activities(base_directory: str, sections: pd.DataFrame) -> pd.DataFrame:    
    return _get_data_for_section(base_directory, sections, get_activities)

def get_assignments(base_directory: str, section_id: int) -> pd.DataFrame:
    file = get_assignments_file(base_directory, section_id)

    return _read_csv(file)

def get_all_assignments(base_directory: str, sections: pd.DataFrame) -> pd.DataFrame:    
    return _get_data_for_section(base_directory, sections, get_assignments)

def get_submissions(base_directory: str, section_id: int, assignment_id: int) -> pd.DataFrame:
    file = get_submissions_file(base_directory, section_id, assignment_id)

    return _read_csv(file)

def get_all_submissions(base_directory: str, assignments: pd.DataFrame) -> pd.DataFrame:    
    df = pd.DataFrame()
    columns = ["SourceSystemIdentifier", "LMSSectionSourceSystemIdentifier"]
    for _, assignment_id, section_id  in assignments[columns].itertuples():
        df = df.append(get_submissions(base_directory, section_id, assignment_id))

    return df

def get_grades(base_directory: str, section_id: int) -> pd.DataFrame:
    file = get_grades_file(base_directory, section_id)

    return _read_csv(file)

def get_all_grades(base_directory: str, sections: pd.DataFrame) -> pd.DataFrame:    
    return _get_data_for_section(base_directory, sections, get_grades)

def get_attendance_events(base_directory: str, section_id: int) -> pd.DataFrame:
    file = get_attendance_events_file(base_directory, section_id)

    return _read_csv(file)

def get_all_attendance_events(base_directory: str, sections: pd.DataFrame) -> pd.DataFrame:    
    return _get_data_for_section(base_directory, sections, get_attendance_events)

## Viewing the Sample Data

To help you understand each of the file formats, below we will load and display a sample of each.

In [8]:
from IPython.display import display, Markdown

sample_dir = os.path.join("..", "..", "docs", "sample-out")

users_df = get_all_users(sample_dir)
display(Markdown("### Users"))
display(users_df.head())

sections_df = get_all_sections(sample_dir)
display(Markdown("### Sections"))
display(sections_df.head())

all_section_associations_df = get_all_section_associations(sample_dir, sections_df)
display(Markdown("### Section Associations"))
display(all_section_associations_df.head())

all_activities_df = get_all_activities(sample_dir, sections_df)
display(Markdown("### Activities"))
display(all_activities_df.head())

all_attendance_events_df = get_all_attendance_events(sample_dir, sections_df)
display(Markdown("### Attendance Events (Schoology only!)"))
display(all_attendance_events_df.head())

all_assignments_df = get_all_assignments(sample_dir, sections_df)
display(Markdown("### Assignments"))
display(all_assignments_df.head())

all_submissions_df = get_all_submissions(sample_dir, all_assignments_df)
display(Markdown("### Submissions"))
display(all_submissions_df.head())

all_grades_df = get_all_grades(sample_dir, sections_df)
display(Markdown("### Grades"))
display(all_grades_df.head())

### Users

,SourceSystemIdentifier,SourceSystem,UserRole,LocalUserIdentifier,SISUserIdentifier,Name,EmailAddress,EntityStatus,CreateDate,LastModifiedDate
0,100032890,Schoology,student,mary.archer,604863,Mary Archer,Mary.Archer@studentgps.org,Archived,8/20/2020 12:34,9/18/2020 12:34
1,100032891,Schoology,student,kyle.hughes,604874,Kyle Hughes,Kyle.Hughes@studentgps.org,Archived,8/20/2020 12:34,9/18/2020 12:34
2,100032892,Schoology,student,peter.nash,604918,Peter Ivan Nash,Peter.Nash@studentgps.org,Active,8/20/2020 12:34,8/20/2020 12:34
3,100032893,Schoology,student,larry.mahoney,604927,Larry Mahoney,Larry.Mahoney@studentgps.org,Active,8/20/2020 12:34,8/20/2020 12:34
4,100032894,Schoology,student,roland.phillips,604938,Roland Phillips,Roland.Phillips@studentgps.org,Active,8/20/2020 12:34,8/20/2020 12:34


### Sections

,SourceSystemIdentifier,SourceSystem,SISSectionIdentifier,Title,SectionDescription,Term,LMSSectionStatus,EntityStatus,CreateDate,LastModifiedDate
0,123456780,Google Classroom,25590100102Trad220ALG112011,ALG-1,Algebra I,"255901001_2020_2019-2020_Fall,255901001_2020_2...",Archived,Active,8/20/2020 12:34,8/20/2020 12:34
1,123456789,Google Classroom,25590100101Trad120ENG112011,ENG-1,English/Language Arts I (9th grade),"255901001_2021_2020-2021_Fall,255901001_2021_2...",Active,Active,8/20/2020 12:34,8/20/2020 12:34
2,2385758954,Google Classroom,25590100102Trad220ALG112011,ALG-1,Algebra I,"255901001_2021_2020-2021_Fall,255901001_2021_2...",Active,Active,8/20/2020 12:34,8/20/2020 12:34
3,123456791,Google Classroom,NaN,ENG-STAFF-1,English language arts staff meeting,NaN,Unpublished,Active,9/3/2020 1:02,9/4/2020 0:01


### Section Associations

...

,SourceSystemIdentifier,SourceSystem,Title,AssignmentCategory,AssignmentDescription,StartDateTime,EndDateTime,DueDateTime,SubmissionType,MaxPoints,LMSSectionSourceSystemIdentifier,EntityStatus,CreateDate,LastModifiedDate
0,2942251001,Schoology,Algebra foundations,Assignment,Complete the Khan Academy unit: Algebra Founda...,2020-08-26 23:59:00,2020-08-28 23:59:00,2020-08-28 23:59:00,online_upload,100,123456780,Active,2020-08-20 12:34:50,2020-08-20 12:34:50
1,2942251002,Schoology,Solving Equations and Inequalities,Assessment,Complete the Khan Academy unit: Solving Equati...,2020-08-28 23:59:00,2020-12-31 23:59:00,2020-12-31 23:59:00,online_quiz,100,123456780,Active,2020-08-20 12:34:50,2020-08-20 12:34:50
0,2942251012,Schoology,Redundancy in War and Peace,Assignment,Describe with statistical examples how Tolsto...,2021-04-01 09:00:00,2021-05-01 00:00:00,2021-05-01 00:00:00,online_upload,100,123456789,Active,2020-08-20 12:34:50,2020-08-20 12:34:50


### Submissions

,SourceSystemIdentifier,SourceSystem,SubmissionStatus,SubmissionDateTime,EarnedPoints,Grade,AssignmentSourceSystemIdentifier,LMSUserSourceSystemIdentifier,EntityStatus,CreateDate,LastModifiedDate
0,456789156,Canvas,Returned,8/23/2020 23:59,77.0,C+,2942251001,2942251012,Active,8/20/2020 12:34,8/20/2020 12:34
1,456789157,Canvas,Returned,12/30/2020 23:59,82.0,B-,2942251001,100032894,Active,8/20/2020 12:34,8/20/2020 12:34
2,456789158,Canvas,Late,2021-05-01 02:00:00,91.0,A-,2942251001,100032897,Active,8/20/2020 12:34,8/20/2020 12:34
3,456789159,Canvas,Created,2021-04-31 04:32:00,NaN,NaN,2942251001,2942251012,Active,8/20/2020 12:34,8/20/2020 12:34
0,456789156,Canvas,Returned,8/23/2020 23:59,77.0,C+,2942251012,2942251012,Active,8/20/2020 12:34,8/20/2020 12:34


### Grades

,SourceSystemIdentifier,SourceSystem,Grade,GradeType,LMSUserLMSSectionAssociationSourceSystemIdentifier,EntityStatus,CreateDate,LastModifiedDate
0,1357982,Canvas,A,Current,8123456,Active,8/20/2020 12:34,8/20/2020 12:34
1,1357983,Canvas,90,Current,8123457,Active,8/20/2020 12:34,8/20/2020 12:34
2,1357984,Canvas,55,Final,8123458,Archived,8/20/2020 12:34,8/20/2020 12:34
3,1357985,Canvas,B,Final,8123459,Active,8/20/2020 12:34,8/20/2020 12:34
0,11357982,Canvas,A,Current,123456,Active,8/20/2020 12:34,8/20/2020 12:34


## Sample Analysis

### In Danger of Failing

Goal: Find all students with average grade (across all sections) below 70, displaying the student name and average grade.

This will require reading the sections file, the users file, section associations, and all grades file. These will need to be joined together, grouped by student, and then the grade will be calculated. Filter by average grade and then display the name and average grade columns.

But there are some "gotchas":

* Some of the grades are alphabetic! Interpret these on a scale like A = 95, B = 85, and so on.
* There are two different grade types: Final and Current. Filter on only Final grades for this excercise.

In [11]:
sample_dir = os.path.join("..", "..", "docs", "sample-out")

users = get_all_users(sample_dir)
sections = get_all_sections(sample_dir)
section_associations = get_all_section_associations(sample_dir, sections)
grades = get_all_grades(sample_dir, sections)

# Merge: users <--> section_associations <--> grades
left_key = "SourceSystemIdentifier"
right_key = "LMSUserSourceSystemIdentifier"
working = users.merge(section_associations, left_on=left_key, right_on=right_key, suffixes=("_u","_sa"))

left_key = "SourceSystemIdentifier_sa"
right_key = "LMSUserLMSSectionAssociationSourceSystemIdentifier"
working = working.merge(grades, left_on=left_key, right_on=right_key, suffixes=("","_g"))

# Filter on Final grades
working = working[working["GradeType"] == "Final"]

# Convert letter grades to numeric grades
def _convert_grades(grade):
    if grade.isnumeric():
        return grade
    
    switch = {
        "A": 95,
        "B": 85,
        "C": 75,
        "D": 65
    }
    return switch.get(grade, 55)

working["Grade"] = working["Grade"].apply(_convert_grades)

# Need to convert the Grade column to a numeric type - it still has the
# generic object type
working["Grade"] = working["Grade"].apply(float)

# Narrow down to columns we care about. For grouping, use the SIS unique 
# identifier instead of the name. The name will be lost in the grouping 
# anyway, and we'll have to re-merge with Users later on to recover it.
working = working[["Name", "SISUserIdentifier", "Grade"]]

# Group by student, using the SIS unique identifier
average_grades = working.groupby(by="SISUserIdentifier", as_index=False).mean(numeric_only=True)

# Pull the student names back in
average_grades = average_grades.merge(users[["Name","SISUserIdentifier"]], on="SISUserIdentifier")

# Filter to those in danger of failing
failing = average_grades[average_grades["Grade"] < 70]

# Finally, display the students of concern
display(Markdown("Students who are currently failing:"))
display(failing)

Students who are currently failing:

,SISUserIdentifier,Grade,Name
1,604927,62.0,Larry Mahoney


### Missing Assignment Submissions

Goal: find students who have not turned in assignment submissions.

Find the assignments that were due before a given date, and then enrich the data with the submission information. Drop records where there _is_ a submission to focus only on those without a submissions. Navigate backward from assignment to section to section association to user to get the students' names.

In [12]:
sample_dir = os.path.join("..", "..", "docs", "sample-out")

users = get_all_users(sample_dir)
sections = get_all_sections(sample_dir)
section_associations = get_all_section_associations(sample_dir, sections)
assignments = get_all_assignments(sample_dir, sections)
submissions = get_all_submissions(sample_dir, assignments)

# The "DueDateTime" column is not automatically recognized as a datetime, so
# convert the data type
assignments["DueDateTime"] = pd.to_datetime(assignments["DueDateTime"])

# Filter assignments by those due before today
from datetime import datetime
assignments = assignments[assignments["DueDateTime"] < datetime.now()]

# Now let's combine some DataFrames to get a collection of all the students who 
# _should_ have turned in an assignment
key = "LMSSectionSourceSystemIdentifier"
working = assignments.merge(section_associations, on=key, suffixes=("","_sa"))

left_key = "LMSUserSourceSystemIdentifier"
right_key = "SourceSystemIdentifier"
working = working.merge(users, left_on=left_key, right_on=right_key, suffixes=("","_u"))

# Join with submissions using an OUTER join so that we keep the Users who do
# not have a submission
left_key = ["SourceSystemIdentifier", "LMSUserSourceSystemIdentifier"]
right_key = ["AssignmentSourceSystemIdentifier", "LMSUserSourceSystemIdentifier"]
working = working.merge(submissions, how="left", left_on=left_key, right_on=right_key, suffixes=("", "_sub"))

# How many rows _do_ have submissions?
count_submitted = working[working["SourceSystemIdentifier_sub"].notnull()].shape[0]
display(Markdown(f"Number of assignments submitted: {count_submitted}"))

# Now drop all rows that _do_ have a submission
no_submission = working[working["SourceSystemIdentifier_sub"].isnull()]

# Might be nice to report the section name too
left_key = "LMSSectionSourceSystemIdentifier"
right_key = "SourceSystemIdentifier"
no_submission = no_submission.merge(sections, left_on=left_key, right_on=right_key, suffixes=("", "_sec"))

# Finally, narrow down to the columns of interest.
no_submission = no_submission[["Name", "Title", "Title_sec"]]
no_submission.rename(columns={"Title_sec": "Section Title", "Title": "Assignment"}, inplace=True)

display(Markdown("Missing assignment submissions:"))
display(no_submission)

Number of assignments submitted: 2

Missing assignment submissions:

,Name,Assignment,Section Title
0,Stephen Caldwell,Algebra foundations,ALG-1
1,Olivia Doris Hardy,Algebra foundations,ALG-1
2,Mary Archer,Algebra foundations,ALG-1
3,Larry Mahoney,Algebra foundations,ALG-1
